In [ ]:
import os
os.chdir('/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_paste import paste_align, paste_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time
from tqdm import tqdm

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-08-07 18:06:05.389795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 18:06:05.553067: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-07 18:06:05.553095: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-07 18:06:06.228849: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [1]:
# Load data
data_folder = "../../data/BARseq/BARseq_Perturbed/"
results_folder = "./results/PASTE/"

In [2]:
subsample_num = 10000
  
for i in tqdm(range(1, 40)):
    slice1 = ad.read_h5ad(filename=os.path.join(data_folder,"slice_{:0>2d}.h5ad".format(i)))
    slice2 = ad.read_h5ad(filename=os.path.join(data_folder,"slice_{:0>2d}.h5ad".format(i+1)))
    
    # subsample
    subsample1 = np.random.choice(slice1.shape[0], subsample_num, replace=False) if slice1.shape[0] > subsample_num else np.arange(slice1.shape[0])
    subsample2 = np.random.choice(slice2.shape[0], subsample_num, replace=False) if slice2.shape[0] > subsample_num else np.arange(slice2.shape[0])
    slice1 = slice1[subsample1,:]
    slice2 = slice2[subsample2,:]

    spatial_key = 'perturbed_spatial'
    key_added = 'align_spatial'
    align_slices, pis = paste_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        device="2",
        verbose=False,
    )
    
    # recover true R and t
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm['perturbed_spatial'])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm['perturbed_spatial'], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1
    results = {"R":R, "t":t, "subsample": [subsample1, subsample2]}
    np.save(os.path.join(results_folder, "slice_{}_{}.npy".format(i,i+1)), results, allow_pickle=True)

  0%|                                                                                                                       | 0/39 [00:00<?, ?it/s]

/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
  3%|██▊                                                                                                            | 1/39 [00:51<32:47, 51.76s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


gpu is available, using gpu.


...

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 37/39 [19:08<00:59, 29.92s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▏  | 38/39 [19:51<00:33, 33.68s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste.py:79: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [20:20<00:00, 31.29s/it]
